In [14]:
import re
import sqlite3

from IPython.core.magic import register_cell_magic
from tabulate import tabulate

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

def clean_sql_block(sql_block):
    IBS = 'INNER_BLOCK_SEPARATOR'
    # Replace semicolons within BEGIN and END blocks
    sql_block = re.sub(r"(BEGIN.*?;.*?END)", lambda m: m.group(0).replace(';', IBS), sql_block, flags=re.DOTALL)
    # Remove comments from -- to the end of the line
    sql_block = re.sub(r"--.*", "", sql_block)

    # Run the normal algorithm
    for s in sql_block.split(';'):
        s = s.replace('\n', ' ')
        s = s.strip()
        if s == '':
            continue

        # Replace the placeholder back to semicolons
        s = s.replace(IBS, ';')

        yield s

def run_and_print(sql):
    try:
        cursor.execute(sql)
    except sqlite3.OperationalError as e:
        print(sql)
        raise e

    conn.commit()

    rows = cursor.fetchall()
    if cursor.description is None:
        return

    print(tabulate(rows, headers=[d[0] for d in cursor.description]), end='\n\n')

@register_cell_magic
def sql(_, cell):
    for sql in clean_sql_block(cell):
        run_and_print(sql)

In [15]:
%%sql

CREATE TABLE debug_logs(message TEXT);
INSERT INTO debug_logs VALUES ('Reticulating splines...');

CREATE TABLE pc_table(pc INTEGER);
INSERT INTO debug_logs VALUES ('pc_table CREATEd');

CREATE TABLE clk_table(clk BOOLEAN);
INSERT INTO debug_logs VALUES ('clk_table CREATEd');

pragma recursive_triggers = 1;


In [5]:
%%sql
-- Insert a new row with a single integer value
INSERT INTO pc_table
    VALUES (1)

In [17]:
%%sql
-- create a trigger that increments the value of pc by 1, up to a maximum of 5, whenever the value of pc is updated

DROP TRIGGER IF EXISTS clk_trigger;

-- CREATE TRIGGER clk_trigger
-- AFTER UPDATE OF clk ON clk_table
-- BEGIN
--     INSERT INTO debug_logs VALUES ('clk_trigger fired');
--     -- Debug out rising and falling edges
--     -- e.g. if transitioning from 0 to 1, print "rising edge", else print "falling edge"
--     INSERT INTO debug_logs VALUES (????);
-- END;
CREATE TRIGGER clk_trigger
AFTER UPDATE OF clk ON clk_table
BEGIN
    INSERT INTO debug_logs VALUES ('clk_trigger fired');

    -- Check for rising or falling edge
    CASE
        WHEN OLD.clk = 0 AND NEW.clk = 1 THEN
            INSERT INTO debug_logs VALUES ('rising edge');
        WHEN OLD.clk = 1 AND NEW.clk = 0 THEN
            INSERT INTO debug_logs VALUES ('falling edge');
    END;
END;

CREATE TRIGGER clk_trigger AFTER UPDATE OF clk ON clk_table BEGIN     INSERT INTO debug_logs VALUES ('clk_trigger fired');           CASE         WHEN OLD.clk = 0 AND NEW.clk = 1 THEN             INSERT INTO debug_logs VALUES ('rising edge');         WHEN OLD.clk = 1 AND NEW.clk = 0 THEN             INSERT INTO debug_logs VALUES ('falling edge');     END


OperationalError: near "CASE": syntax error

In [10]:
%%sql
-- update the value of pc to 2
UPDATE pc_table
SET pc = 2

UPDATE pc_table SET pc = 2


OperationalError: too many levels of trigger recursion

In [ ]:
%%sql

-- retrieve and print the data
SELECT * FROM pc_table;

-- retrieve and print the logs
SELECT * FROM debug_logs;

  pc
----
1002
   1
message
-----------------------
Reticulating splines...
pc_table CREATEd
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger started
Trigger sta

In [ ]:
# Commit the changes and close the connection
conn.commit()
conn.close()